In [31]:
import os
import pickle
import posixpath
import argparse
import time
import io
import threading
import json
import re
import stat
import math
import traceback
import gzip
import sys

import urllib
import urllib.parse
import wsgiref.handlers

import tqdm

In [1]:
game_obj_list_filepath = "modified_game_obj_list-1653552476412252800.pickle"
game_binary_url_list_filepath = "tmp_pickle_files/game_binary_url_list-1650280741999342000.pickle"
image_url_info_dict_filepath = "image_url_info_dict-1654534265407578600.pickle"

In [23]:
game_obj_list = pickle.load(open(game_obj_list_filepath, 'rb'))
game_binary_url_list = pickle.load(open(game_binary_url_list_filepath, 'rb'))
image_url_info_dict = pickle.load(open(image_url_info_dict_filepath, 'rb'))

In [5]:
len(game_obj_list)

11732

In [7]:
game_obj_list[0]

{'url': 'http://www.7723.cn/download/10819.htm',
 'name': '星际车手',
 'banner_image': {'url': 'http://image.7723.cn/wuza/picp/201215O5239542.jpg',
  'url_hash': '1b22470012a9e4c4680e90dbe361bb67'},
 'gameplay_image_list': [{'url': 'http://image.7723.cn/wuza/picp/201215O5239542.jpg',
   'url_hash': '1b22470012a9e4c4680e90dbe361bb67'},
  {'url': 'http://images.7723.cn/wuza/picp/201215O5239522.gif',
   'url_hash': '982ff0d035346044ebc0158d297a5ddf'},
  {'url': 'http://images.7723.cn/wuza/picp/201215O5242894.gif',
   'url_hash': 'c47d55d8f084e86d253737b46517fa91'}],
 'binary_info_list': [{'url': 'http://www.7723.cn/downb.asp?idd=13&id=10819&ksp=1',
   'description': '诺基亚 N73系列(240×320)\nN73 5320 5320XM 5320di_XM 5630XM 5700 5700XM 5710XM 5730XM 6110 6110N 6120 6120C 6120ci 6121 6122C 6124C 6210S 6210ci 6220C 6290 6650F 6700S 6702S 6710N 6720C 6730c 6788 6788I 6790 C5 C5-01 E101 E50 E51 E52 E55 E65 E66 E75 X5-00 X5-01 N71 N73ie N75 N76 N76-1 N77 N78 N79 N79 Eco N81 N81 8GB N82 N85 N86 N92 N93 

In [16]:
game_info_with_no_images_in_cache_list = []
game_info_with_some_images_in_cache_list = []

for i in tqdm.tqdm(range(len(game_obj_list))):
    has_no_images_in_cache = True
    has_all_images_in_cache = True

    number_of_images_in_cache = 0
    number_of_images = 0

    # banner_image
    if 'banner_image' in game_obj_list[i]:
        if game_obj_list[i]['banner_image'] is not None:
            number_of_images += 1
            url = game_obj_list[i]['banner_image']['url']
            in_cache = (url in image_url_info_dict)
            game_obj_list[i]['banner_image']['in_cache'] = in_cache
            if in_cache:
                has_no_images_in_cache = False
                number_of_images_in_cache += 1
            else:
                has_all_images_in_cache = False

    # gameplay_image_list
    if 'gameplay_image_list' in game_obj_list[i]:
        for gameplay_image in game_obj_list[i]['gameplay_image_list']:
            number_of_images += 1
            url = gameplay_image['url']
            in_cache = (url in image_url_info_dict)
            gameplay_image['in_cache'] = in_cache
            if in_cache:
                number_of_images_in_cache += 1
                has_no_images_in_cache = False
            else:
                has_all_images_in_cache = False

    if has_no_images_in_cache:
        game_info_with_no_images_in_cache_list.append(game_obj_list[i])
    elif not has_all_images_in_cache:
        game_obj_list[i]['number_of_images_note'] = f'{number_of_images_in_cache}/{number_of_images}'
        game_info_with_some_images_in_cache_list.append(game_obj_list[i])


100%|████████████████████████████████████████████████████████████████████████| 11732/11732 [00:00<00:00, 187772.89it/s]


In [17]:
len(game_info_with_no_images_in_cache_list)

4751

In [18]:
len(game_info_with_some_images_in_cache_list)

36

In [19]:
len(game_info_with_no_images_in_cache_list) + len(game_info_with_some_images_in_cache_list)

4787

In [20]:
len(game_obj_list)

11732

In [21]:
for game_info in game_info_with_some_images_in_cache_list:
    print(game_info['name'], game_info['number_of_images_note'])

米老鼠西游记 2/5
数独 1/3
3D虚拟宠物 1/3
惊愿破解版 1/3
彩虹城之剑-驯龙勇者 1/3
魂斗罗执念中文版[GBA] 1/3
温布尔登网球公开赛2009豪华版 1/3
星战纪元 1/3
古墓丽影传奇3D 1/3
钻石岛汉化版 2/4
韩版疯狂坦克 1/3
滑行因子 1/3
魔兽争霸Stratagus2.1 1/3
德克萨斯扑克豪华版 1/3
移动小狗(摄像头游戏) 1/3
天魂殇2云起龙襄 5/7
3D江河风暴 1/3
街霸2011勇者无敌 4/6
反恐特警3 1/3
淘金者汉化版 2/3
我的模拟人生 2/4
次世代美少女战士 1/3
公主台球之换装俱乐部 1/3
神秘公寓弹球 1/3
血猎传说 1/3
战斧[MD] 2/3
疯狂涂鸦 2/5
实况篮球2009 1/3
金刚[GBA] 1/3
三国志-英雄无敌 1/3
猿人武士-横版过关 1/3
斗地主之棋牌大奖赛 4/6
功夫-雄霸天下 1/3
街头篮球[GBA] 1/3
摇滚吉他之旅2 1/3
CS反恐特警真3D版 1/3


In [22]:
game_info_with_no_images_in_cache_list[0]

{'url': 'http://www.7723.cn/download/1737.htm',
 'name': '雷曼IV',
 'banner_image': {'url': 'http://image.7723.cn/admin/wzpic/200611292128576299.gif',
  'url_hash': 'c5eb6e687a80ef5b1d069f08deb05610',
  'in_cache': False},
 'gameplay_image_list': [{'url': 'http://image.7723.cn/admin/wzpic/200611292128576299.gif',
   'url_hash': 'c5eb6e687a80ef5b1d069f08deb05610',
   'in_cache': False},
  {'url': 'http://images.7723.cn/admin/wzpic/2006112921287965618.gif',
   'url_hash': '0a1dd3d96e08a4935b74e7c83bd89a36',
   'in_cache': False}],
 'binary_info_list': [{'url': 'http://www.7723.cn/downb.asp?idd=7&id=1737&ksp=1',
   'description': '诺基亚 N73系列(240×320)\nN73 5320 5320XM 5320di_XM 5630XM 5700 5700XM 5710XM 5730XM 6110 6110N 6120 6120C 6120ci 6121 6122C 6124C 6210S 6210ci 6220C 6290 6650F 6700S 6702S 6710N 6720C 6730c 6788 6788I 6790 C5 C5-01 E101 E50 E51 E52 E55 E65 E66 E75 X5-00 X5-01 N71 N73ie N75 N76 N76-1 N77 N78 N79 N79 Eco N81 N81 8GB N82 N85 N86 N92 N93 N93I N95 N958G N95 8GB N95-3 NAM N9

In [24]:
for i in tqdm.tqdm(range(len(game_obj_list))):
    # banner_image
    if 'banner_image' in game_obj_list[i]:
        if game_obj_list[i]['banner_image'] is not None:
            number_of_images += 1
            url = game_obj_list[i]['banner_image']['url']
            in_cache = (url in image_url_info_dict)
            game_obj_list[i]['banner_image']['in_cache'] = in_cache

    # gameplay_image_list
    if 'gameplay_image_list' in game_obj_list[i]:
        for gameplay_image in game_obj_list[i]['gameplay_image_list']:
            number_of_images += 1
            url = gameplay_image['url']
            in_cache = (url in image_url_info_dict)
            gameplay_image['in_cache'] = in_cache

100%|████████████████████████████████████████████████████████████████████████| 11732/11732 [00:00<00:00, 175211.32it/s]


In [25]:
log_filepath = f'game_info_list-{time.time_ns()}.pickle'
print(f'log_filepath: {log_filepath}')
with open(log_filepath, 'wb') as outfile:
    pickle.dump(game_obj_list, outfile)

log_filepath: game_info_list-1654801537767092400.pickle


In [28]:
filesize = os.path.getsize(log_filepath)
print(f'{filesize} bytes')
print(f'{(filesize / 1024 / 1024):.02f} MB')

57067255 bytes
54.42 MB


In [30]:
log_filepath = f'game_info_list-{time.time_ns()}.pickle.gzip'
print(f'log_filepath: {log_filepath}')
with gzip.open(log_filepath, 'wb') as outfile:
    pickle.dump(game_obj_list, outfile)

filesize = os.path.getsize(log_filepath)
print(f'{filesize} bytes')
print(f'{(filesize / 1024 / 1024):.02f} MB')

log_filepath: game_info_list-1654801782759906900.pickle.gzip
9142397 bytes
8.72 MB


In [33]:
sys.getsizeof(game_obj_list)

105696

In [34]:
game_obj_list[0]

{'url': 'http://www.7723.cn/download/10819.htm',
 'name': '星际车手',
 'banner_image': {'url': 'http://image.7723.cn/wuza/picp/201215O5239542.jpg',
  'url_hash': '1b22470012a9e4c4680e90dbe361bb67',
  'in_cache': True},
 'gameplay_image_list': [{'url': 'http://image.7723.cn/wuza/picp/201215O5239542.jpg',
   'url_hash': '1b22470012a9e4c4680e90dbe361bb67',
   'in_cache': True},
  {'url': 'http://images.7723.cn/wuza/picp/201215O5239522.gif',
   'url_hash': '982ff0d035346044ebc0158d297a5ddf',
   'in_cache': True},
  {'url': 'http://images.7723.cn/wuza/picp/201215O5242894.gif',
   'url_hash': 'c47d55d8f084e86d253737b46517fa91',
   'in_cache': True}],
 'binary_info_list': [{'url': 'http://www.7723.cn/downb.asp?idd=13&id=10819&ksp=1',
   'description': '诺基亚 N73系列(240×320)\nN73 5320 5320XM 5320di_XM 5630XM 5700 5700XM 5710XM 5730XM 6110 6110N 6120 6120C 6120ci 6121 6122C 6124C 6210S 6210ci 6220C 6290 6650F 6700S 6702S 6710N 6720C 6730c 6788 6788I 6790 C5 C5-01 E101 E50 E51 E52 E55 E65 E66 E75 X5-00